In [7]:
import pandas as pd
from youtube_crawler import *
import json
import sys
import csv
import re
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import json, time

In [47]:
################
#write the crawler
################

%%writefile youtube_crawler.py

# Reference: https://stackoverflow.com/a/57076690/15164646
from selenium import webdriver
import json, time 

def get_videos(ticker):
    driver = webdriver.Chrome("/Users/miao/Desktop/chromedriver") #add path to webdriver
    driver.get('https://www.youtube.com/results?search_query={}+stock'.format(ticker))

    youtube_data = []

    # scrolling to the end of the page
    while True:
        # end_result = "No more results" string at the bottom of the page
        # this will be used to break out of the while loop
        end_result = driver.find_element_by_css_selector('#message').is_displayed()
        driver.execute_script("var scrollingElement = (document.scrollingElement || document.body);scrollingElement.scrollTop = scrollingElement.scrollHeight;")
        #print(end_result)

        if end_result == True:
            break

    print('Extracting results for {}. It might take a while...'.format(ticker))

    for result in driver.find_elements_by_css_selector('.text-wrapper.style-scope.ytd-video-renderer'):
        title = result.find_element_by_css_selector('.title-and-badge.style-scope.ytd-video-renderer').text
        link = result.find_element_by_css_selector('.title-and-badge.style-scope.ytd-video-renderer a').get_attribute('href')
        channel_name = result.find_element_by_css_selector('.long-byline').text
        channel_link = result.find_element_by_css_selector('#text > a').get_attribute('href')
        views = result.find_element_by_css_selector('.style-scope ytd-video-meta-block').text.split('\n')[0]

        try:
            time_published = result.find_element_by_css_selector('.style-scope ytd-video-meta-block').text.split('\n')[1]
        except:
            time_published = None

        try:
            snippet = result.find_element_by_css_selector('.metadata-snippet-container').text
        except:
            snippet = None

        try:
            if result.find_element_by_css_selector('#channel-name .ytd-badge-supported-renderer') is not None:
                verified_badge = True
            else:
                verified_badge = False
        except:
            verified_badge = None

        try:
            extensions = result.find_element_by_css_selector('#badges .ytd-badge-supported-renderer').text
        except:
            extensions = None
        #print(verified_badge)
        
        #threshold for views above 1000
        if "K" in views or "M" in views:
            youtube_data.append({
                'title': title,
                'link': link,
                'channel': {'channel_name': channel_name, 'channel_link': channel_link},
                'views': views,
                'time_published': time_published,
                'snippet': snippet,
                'verified_badge': verified_badge,
                'extensions': extensions,
            })
        
    print("crawled", len(youtube_data), "videos")
    #print(json.dumps(youtube_data, indent=2, ensure_ascii=False))
    driver.quit()
    
    return youtube_data


In [67]:
#################
#read the SP500 list data
#################
df = pd.read_csv('SP500_List.csv')

In [32]:
#################
#Start scraping
#################
video_lst = {}
channel_list = []

for index, row in df.iterrows():
    a = time.time()
    ticker, noise, name = row[0], row[1], row[2]
    if noise == 0:
        search = ticker
    else:
        search = name
    
    videos = get_videos(search)
    video_lst[row[0]] = videos #store raw datat in dict
    
    name_list = []
    #build a video table
    print("building channel table for {}".format(ticker))
    for video in videos:
        channel = video['channel']['channel_name']
        link = video['channel']['channel_link']
        if channel not in name_list:
            name_list.append(channel)
            channel_list.append([ticker, name, noise, channel, link])    
        if len(name_list) >= 20:
            break
    
    b = time.time()
    print(index+1, 'finished')
    print('time used:', b-a)

Extracting results for Agilent Technologies. It might take a while...
crawled 224 videos
building channel table for A
1 finished
time used: 239.712628364563
Extracting results for AAL. It might take a while...
crawled 219 videos
building channel table for AAL
2 finished
time used: 256.67175006866455
Extracting results for AAP. It might take a while...
crawled 360 videos
building channel table for AAP
3 finished
time used: 274.62680196762085
Extracting results for AAPL. It might take a while...
crawled 269 videos
building channel table for AAPL
4 finished
time used: 248.96890091896057
Extracting results for ABBV. It might take a while...
crawled 292 videos
building channel table for ABBV
5 finished
time used: 250.10623383522034
Extracting results for AmerisourceBergen. It might take a while...
crawled 233 videos
building channel table for ABC
6 finished
time used: 216.6250400543213
Extracting results for ABMD. It might take a while...
crawled 220 videos
building channel table for ABMD
7

KeyboardInterrupt: 

In [ ]:
##################
#Perhaps because of anti-scraping machenisms of youtube, the scraping process was disrupted several times and I had to rerun.
#Please see the rest of the scraping process in the last part of the script
#################

In [78]:
#Below are the data output process

#################
#output raw_videos to json
#################
with open("raw_videos.json", "w") as outfile:
    json.dump(video_lst, outfile)

In [111]:
#check if all videos are scraped
len(video_lst)

505

In [113]:
###################
#convert channel list to data frame: ticker - stock name - noise - channel - link
###################

df_channel = pd.DataFrame(channel_list, columns = ['Ticker', 'Stock_Name', 'noise', 'Channel_Name', 'Channel_link'])
#check if all tickers are scraped
df_channel.tail()

,Ticker,Stock_Name,noise,Channel_Name,Channel_link
10095,ZTS,Zoetis,0,IPO Market Watch,https://www.youtube.com/channel/UCQ-ycmYXc_1B-...
10096,ZTS,Zoetis,0,Will Knowledge,https://www.youtube.com/channel/UCXnjHTVPeCp7h...
10097,ZTS,Zoetis,0,Jerry Romine Entrepreneur Abroad,https://www.youtube.com/user/padnag1
10098,ZTS,Zoetis,0,Trading Jesus,https://www.youtube.com/channel/UCNYfwelOr7Hax...
10099,ZTS,Zoetis,0,MagnatesMedia,https://www.youtube.com/channel/UCE4Gn00XZbpWv...


In [114]:
#output channel dataframe to csv
df_channel.to_csv("Ticker_Channel.csv", encoding='utf-8', index=False)

In [97]:
######################
#create a table mapping tickers to channel: channel - link - ticker - stock name
######################
ticker_to_channel = {}
channels = []
for index, row in df_channel.iterrows():
    channel = row[3]
    if channel not in channels:
        ticker_to_channel[channel] = [row[4], [row[0]], [row[1]]]
        channels.append(channel)
    else:
        ticker_to_channel[channel][1].append(row[0])
        ticker_to_channel[channel][2].append(row[1])

In [110]:
#output to csv
df_ticker_to_channel = pd.DataFrame(ticker_to_channel).T #columns = ['Channel_Name', 'Channel_Link', 'Ticker'])
df_ticker_to_channel.reset_index(level=0, inplace=True)
df_ticker_to_channel.columns = ['Channel_Name', 'Channel_Link', 'Ticker', 'Stock_Name']
df_ticker_to_channel
df_ticker_to_channel.to_csv("Channel_Ticker.csv", encoding='utf-8', index=False)

In [112]:
df_ticker_to_channel

,Channel_Name,Channel_Link,Ticker,Stock_Name
0,Agilent Technologies,https://www.youtube.com/user/agilent,[A],[Agilent Technologies]
1,"Value Investing with Sven Carlin, Ph.D.",https://www.youtube.com/channel/UCrTTBSUr0zhPU...,"[A, ABBV, ABC, ABT, ADM, ADP, AES, ALB, AMGN, ...","[Agilent Technologies, AbbVie, AmerisourceBerg..."
2,CNBC Television,https://www.youtube.com/channel/UCrp_UI8XtuYfp...,"[A, AAL, AAPL, ABBV, ABC, ABMD, ABT, ACN, ADBE...","[Agilent Technologies, American Airlines Group..."
3,The Average Joe Investor,https://www.youtube.com/channel/UCN9_N18CDKFh2...,"[A, AMCR, BLK, CINF, CLX, ETN, EXPD, GPC, GWW,...","[Agilent Technologies, Amcor, BlackRock, Cinci..."
4,Star,https://www.youtube.com/channel/UCuPnGgHJzIJ2t...,"[A, ACN, ANSS, APD, AVGO, BMY, CCI, CDNS, CNP,...","[Agilent Technologies, Accenture, Ansys, Air P..."
...,...,...,...,...
3246,SG NINJAS LLC,https://www.youtube.com/user/xxjavasxx,[ZTS],[Zoetis]
3247,90's生活投資頻道 - 史蒂夫&戴夫,https://www.youtube.com/channel/UCRuHb5JUKrBub...,[ZTS],[Zoetis]
3248,noizx0,https://www.youtube.com/user/noizx0,[ZTS],[Zoetis]
3249,Scramel,https://www.youtube.com/channel/UC3d2oC_tquc8R...,[ZTS],[Zoetis]


In [44]:
################
#Please see the rest of the scraping process here
################

###stuck in 36, start from ANSS
for index, row in df.iterrows():
    if index >= 36:
        a = time.time()
        ticker, noise, name = row[0], row[1], row[2]
        if noise == 0:
            search = ticker
        else:
            search = name

        videos = get_videos(search)
        video_lst[row[0]] = videos #store raw datat in dict

        name_list = []
        #build a video table
        print("building channel table for {}".format(ticker))
        for video in videos:
            channel = video['channel']['channel_name']
            link = video['channel']['channel_link']
            if channel not in name_list:
                name_list.append(channel)
                channel_list.append([ticker, name, noise, channel, link])    
            if len(name_list) >= 20:
                break

        b = time.time()
        print(index+1, 'finished')
        print('time used:', b-a)

Extracting results for ANSS. It might take a while...
crawled 235 videos
building channel table for ANSS
37 finished
time used: 213.67900109291077
Extracting results for ANTM. It might take a while...
crawled 421 videos
building channel table for ANTM
38 finished
time used: 272.85671186447144
Extracting results for AON. It might take a while...
crawled 472 videos
building channel table for AON
39 finished
time used: 258.83991289138794
Extracting results for AOS. It might take a while...
crawled 328 videos
building channel table for AOS
40 finished
time used: 269.92253708839417
Extracting results for APA. It might take a while...
crawled 439 videos
building channel table for APA
41 finished
time used: 282.98853611946106
Extracting results for APD. It might take a while...
crawled 466 videos
building channel table for APD
42 finished
time used: 274.4673330783844
Extracting results for APH. It might take a while...
crawled 335 videos
building channel table for APH
43 finished
time used: 2

KeyboardInterrupt: 

In [45]:
###stuck in 78, start from CARR
for index, row in df.iterrows():
    if index >= 78:
        a = time.time()
        ticker, noise, name = row[0], row[1], row[2]
        if noise == 0:
            search = ticker
        else:
            search = name

        videos = get_videos(search)
        video_lst[row[0]] = videos #store raw datat in dict

        name_list = []
        #build a video table
        print("building channel table for {}".format(ticker))
        for video in videos:
            channel = video['channel']['channel_name']
            link = video['channel']['channel_link']
            if channel not in name_list:
                name_list.append(channel)
                channel_list.append([ticker, name, noise, channel, link])    
            if len(name_list) >= 20:
                break

        b = time.time()
        print(index+1, 'finished')
        print('time used:', b-a)

Extracting results for CARR. It might take a while...
crawled 386 videos
building channel table for CARR
79 finished
time used: 234.8265221118927
Extracting results for CAT. It might take a while...
crawled 305 videos
building channel table for CAT
80 finished
time used: 365.8545801639557
Extracting results for CB. It might take a while...
crawled 356 videos
building channel table for CB
81 finished
time used: 249.53437209129333
Extracting results for CBOE. It might take a while...
crawled 291 videos
building channel table for CBOE
82 finished
time used: 288.6092121601105
Extracting results for CBRE. It might take a while...
crawled 247 videos
building channel table for CBRE
83 finished
time used: 253.47239995002747
Extracting results for Crown Castle. It might take a while...
crawled 328 videos
building channel table for CCI
84 finished
time used: 268.78692388534546
Extracting results for CCL. It might take a while...
crawled 224 videos
building channel table for CCL
85 finished
time 

KeyboardInterrupt: 

In [52]:
###stuck in 123, start from CVS
for index, row in df.iterrows():
    if index >= 123:
        a = time.time()
        ticker, noise, name = row[0], row[1], row[2]
        if noise == 0:
            search = ticker
        else:
            search = name

        videos = get_videos(search)
        video_lst[row[0]] = videos #store raw datat in dict

        name_list = []
        #build a video table
        print("building channel table for {}".format(ticker))
        for video in videos:
            channel = video['channel']['channel_name']
            link = video['channel']['channel_link']
            if channel not in name_list:
                name_list.append(channel)
                channel_list.append([ticker, name, noise, channel, link])    
            if len(name_list) >= 20:
                break

        b = time.time()
        print(index+1, 'finished')
        print('time used:', b-a)

Extracting results for CVS. It might take a while...
crawled 391 videos
building channel table for CVS
124 finished
time used: 249.75520086288452
Extracting results for CVX. It might take a while...
crawled 377 videos
building channel table for CVX
125 finished
time used: 273.99754905700684
Extracting results for CZR. It might take a while...
crawled 393 videos
building channel table for CZR
126 finished
time used: 298.43533277511597
Extracting results for Dominion Energy. It might take a while...
crawled 357 videos
building channel table for D
127 finished
time used: 943.8370952606201
Extracting results for DAL. It might take a while...
crawled 391 videos
building channel table for DAL
128 finished
time used: 339.78088307380676
Extracting results for DuPont. It might take a while...
crawled 337 videos
building channel table for DD
129 finished
time used: 278.3126947879791
Extracting results for Deere & Co.. It might take a while...
crawled 607 videos
building channel table for DE
130 

Extracting results for FB. It might take a while...
crawled 301 videos
building channel table for FB
180 finished
time used: 276.72573494911194
Extracting results for FBHS. It might take a while...
crawled 231 videos
building channel table for FBHS
181 finished
time used: 219.05434393882751
Extracting results for FCX. It might take a while...
crawled 341 videos
building channel table for FCX
182 finished
time used: 249.6337730884552
Extracting results for FDX. It might take a while...
crawled 307 videos
building channel table for FDX
183 finished
time used: 268.94612884521484
Extracting results for FE. It might take a while...
crawled 318 videos
building channel table for FE
184 finished
time used: 241.66094994544983
Extracting results for FFIV. It might take a while...
crawled 155 videos
building channel table for FFIV
185 finished
time used: 213.15870904922485
Extracting results for FIS. It might take a while...
crawled 396 videos
building channel table for FIS
186 finished
time used

Extracting results for IEX. It might take a while...
crawled 228 videos
building channel table for IEX
236 finished
time used: 204.59295201301575
Extracting results for IFF. It might take a while...
crawled 421 videos
building channel table for IFF
237 finished
time used: 198.01402807235718
Extracting results for ILMN. It might take a while...
crawled 258 videos
building channel table for ILMN
238 finished
time used: 220.39289021492004
Extracting results for INCY. It might take a while...
crawled 297 videos
building channel table for INCY
239 finished
time used: 195.23192286491394
Extracting results for INFO. It might take a while...


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=94.0.4606.71)


In [57]:
#Stuck in 239, start from INFO
for index, row in df.iterrows():
    if index >= 239:
        a = time.time()
        ticker, noise, name = row[0], row[1], row[2]
        if noise == 0:
            search = ticker
        else:
            search = name

        videos = get_videos(search)
        video_lst[row[0]] = videos #store raw datat in dict

        name_list = []
        #build a video table
        print("building channel table for {}".format(ticker))
        for video in videos:
            channel = video['channel']['channel_name']
            link = video['channel']['channel_link']
            if channel not in name_list:
                name_list.append(channel)
                channel_list.append([ticker, name, noise, channel, link])    
            if len(name_list) >= 20:
                break

        b = time.time()
        print(index+1, 'finished')
        print('time used:', b-a)

Extracting results for IHS Markit. It might take a while...
crawled 274 videos
building channel table for INFO
240 finished
time used: 315.16393280029297
Extracting results for INTC. It might take a while...
crawled 364 videos
building channel table for INTC
241 finished
time used: 278.10277676582336
Extracting results for INTU. It might take a while...
crawled 311 videos
building channel table for INTU
242 finished
time used: 241.3235158920288
Extracting results for IP. It might take a while...
crawled 423 videos
building channel table for IP
243 finished
time used: 236.37776017189026
Extracting results for IPG. It might take a while...
crawled 320 videos
building channel table for IPG
244 finished
time used: 233.11341309547424
Extracting results for IPGP. It might take a while...
crawled 220 videos
building channel table for IPGP
245 finished
time used: 216.16242265701294
Extracting results for IQV. It might take a while...
crawled 233 videos
building channel table for IQV
246 finish

Extracting results for MCD. It might take a while...
crawled 351 videos
building channel table for MCD
296 finished
time used: 233.21963596343994
Extracting results for MCHP. It might take a while...
crawled 224 videos
building channel table for MCHP
297 finished
time used: 234.5729238986969
Extracting results for MCK. It might take a while...
crawled 400 videos
building channel table for MCK
298 finished
time used: 175.98995423316956
Extracting results for MCO. It might take a while...
crawled 410 videos
building channel table for MCO
299 finished
time used: 246.79740691184998
Extracting results for MDLZ. It might take a while...
crawled 154 videos
building channel table for MDLZ
300 finished
time used: 237.53400588035583
Extracting results for MDT. It might take a while...
crawled 450 videos
building channel table for MDT
301 finished
time used: 236.84180116653442
Extracting results for MET. It might take a while...
crawled 316 videos
building channel table for MET
302 finished
time 

Extracting results for OMC. It might take a while...
crawled 467 videos
building channel table for OMC
352 finished
time used: 276.62925910949707
Extracting results for ORCL. It might take a while...
crawled 287 videos
building channel table for ORCL
353 finished
time used: 298.89606499671936
Extracting results for ORLY. It might take a while...
crawled 518 videos
building channel table for ORLY
354 finished
time used: 282.4800503253937
Extracting results for OTIS. It might take a while...
crawled 412 videos
building channel table for OTIS
355 finished
time used: 201.70848679542542
Extracting results for OXY. It might take a while...
crawled 411 videos
building channel table for OXY
356 finished
time used: 262.1198000907898
Extracting results for PAYC. It might take a while...
crawled 211 videos
building channel table for PAYC
357 finished
time used: 241.7855019569397
Extracting results for PAYX. It might take a while...
crawled 246 videos
building channel table for PAYX
358 finished
t

WebDriverException: Message: disconnected: not connected to DevTools
  (Session info: chrome=94.0.4606.71)


In [59]:
#disconnected, start from 399
for index, row in df.iterrows():
    if index >= 399:
        a = time.time()
        ticker, noise, name = row[0], row[1], row[2]
        if noise == 0:
            search = ticker
        else:
            search = name

        videos = get_videos(search)
        video_lst[row[0]] = videos #store raw datat in dict

        name_list = []
        #build a video table
        print("building channel table for {}".format(ticker))
        for video in videos:
            channel = video['channel']['channel_name']
            link = video['channel']['channel_link']
            if channel not in name_list:
                name_list.append(channel)
                channel_list.append([ticker, name, noise, channel, link])    
            if len(name_list) >= 20:
                break

        b = time.time()
        print(index+1, 'finished')
        print('time used:', b-a)

Extracting results for ROK. It might take a while...
crawled 385 videos
building channel table for ROK
400 finished
time used: 280.05841302871704
Extracting results for ROL. It might take a while...
crawled 395 videos
building channel table for ROL
401 finished
time used: 173.96103620529175
Extracting results for ROP. It might take a while...
crawled 409 videos
building channel table for ROP
402 finished
time used: 178.5984160900116
Extracting results for ROST. It might take a while...
crawled 383 videos
building channel table for ROST
403 finished
time used: 193.1280870437622
Extracting results for RSG. It might take a while...
crawled 304 videos
building channel table for RSG
404 finished
time used: 231.1334137916565
Extracting results for RTX. It might take a while...
crawled 358 videos
building channel table for RTX
405 finished
time used: 248.54555702209473
Extracting results for SBAC. It might take a while...
crawled 233 videos
building channel table for SBAC
406 finished
time us

KeyboardInterrupt: 

In [63]:
#disconnected, start from 419 STE
for index, row in df.iterrows():
    if index >= 419:
        a = time.time()
        ticker, noise, name = row[0], row[1], row[2]
        if noise == 0:
            search = ticker
        else:
            search = name

        videos = get_videos(search)
        video_lst[row[0]] = videos #store raw datat in dict

        name_list = []
        #build a video table
        print("building channel table for {}".format(ticker))
        for video in videos:
            channel = video['channel']['channel_name']
            link = video['channel']['channel_link']
            if channel not in name_list:
                name_list.append(channel)
                channel_list.append([ticker, name, noise, channel, link])    
            if len(name_list) >= 20:
                break

        b = time.time()
        print(index+1, 'finished')
        print('time used:', b-a)

Extracting results for STE. It might take a while...
crawled 403 videos
building channel table for STE
420 finished
time used: 352.3815231323242
Extracting results for STT. It might take a while...
crawled 362 videos
building channel table for STT
421 finished
time used: 209.3703751564026
Extracting results for STX. It might take a while...
crawled 434 videos
building channel table for STX
422 finished
time used: 257.1139268875122
Extracting results for STZ. It might take a while...
crawled 377 videos
building channel table for STZ
423 finished
time used: 221.2283570766449
Extracting results for SWK. It might take a while...
crawled 361 videos
building channel table for SWK
424 finished
time used: 235.7377688884735
Extracting results for SWKS. It might take a while...
crawled 217 videos
building channel table for SWKS
425 finished
time used: 209.40493416786194
Extracting results for SYF. It might take a while...
crawled 258 videos
building channel table for SYF
426 finished
time used: 

KeyboardInterrupt: 

In [69]:
#disconnected, start from 428 T
for index, row in df.iterrows():
    if index >= 428:
        a = time.time()
        ticker, noise, name = row[0], row[1], row[2]
        if noise == 0:
            search = ticker
        else:
            search = name

        videos = get_videos(search)
        video_lst[row[0]] = videos #store raw datat in dict

        name_list = []
        #build a video table
        print("building channel table for {}".format(ticker))
        for video in videos:
            channel = video['channel']['channel_name']
            link = video['channel']['channel_link']
            if channel not in name_list:
                name_list.append(channel)
                channel_list.append([ticker, name, noise, channel, link])    
            if len(name_list) >= 20:
                break

        b = time.time()
        print(index+1, 'finished')
        print('time used:', b-a)

Extracting results for T. It might take a while...
crawled 432 videos
building channel table for T
429 finished
time used: 379.7960331439972
Extracting results for TAP. It might take a while...
crawled 358 videos
building channel table for TAP
430 finished
time used: 276.97210001945496
Extracting results for TDG. It might take a while...
crawled 346 videos
building channel table for TDG
431 finished
time used: 210.47032713890076
Extracting results for TDY. It might take a while...
crawled 394 videos
building channel table for TDY
432 finished
time used: 236.56955885887146
Extracting results for Bio-Techne. It might take a while...
crawled 244 videos
building channel table for TECH
433 finished
time used: 230.386638879776
Extracting results for TEL. It might take a while...
crawled 228 videos
building channel table for TEL
434 finished
time used: 245.77825498580933
Extracting results for TER. It might take a while...
crawled 415 videos
building channel table for TER
435 finished
time us

KeyboardInterrupt: 

In [70]:
#disconnected, start from 447 TSN
for index, row in df.iterrows():
    if index >= 447:
        a = time.time()
        ticker, noise, name = row[0], row[1], row[2]
        if noise == 0:
            search = ticker
        else:
            search = name

        videos = get_videos(search)
        video_lst[row[0]] = videos #store raw datat in dict

        name_list = []
        #build a video table
        print("building channel table for {}".format(ticker))
        for video in videos:
            channel = video['channel']['channel_name']
            link = video['channel']['channel_link']
            if channel not in name_list:
                name_list.append(channel)
                channel_list.append([ticker, name, noise, channel, link])    
            if len(name_list) >= 20:
                break

        b = time.time()
        print(index+1, 'finished')
        print('time used:', b-a)

Extracting results for TSN. It might take a while...
crawled 392 videos
building channel table for TSN
448 finished
time used: 198.17190313339233
Extracting results for TT. It might take a while...
crawled 278 videos
building channel table for TT
449 finished
time used: 212.70503520965576


KeyboardInterrupt: 

In [71]:
#disconnected, start from 449 TTWO
for index, row in df.iterrows():
    if index >= 449:
        a = time.time()
        ticker, noise, name = row[0], row[1], row[2]
        if noise == 0:
            search = ticker
        else:
            search = name

        videos = get_videos(search)
        video_lst[row[0]] = videos #store raw datat in dict

        name_list = []
        #build a video table
        print("building channel table for {}".format(ticker))
        for video in videos:
            channel = video['channel']['channel_name']
            link = video['channel']['channel_link']
            if channel not in name_list:
                name_list.append(channel)
                channel_list.append([ticker, name, noise, channel, link])    
            if len(name_list) >= 20:
                break

        b = time.time()
        print(index+1, 'finished')
        print('time used:', b-a)

Extracting results for TTWO. It might take a while...
crawled 204 videos
building channel table for TTWO
450 finished
time used: 339.1453061103821
Extracting results for TWTR. It might take a while...
crawled 240 videos
building channel table for TWTR
451 finished
time used: 250.0038378238678
Extracting results for TXN. It might take a while...
crawled 484 videos
building channel table for TXN
452 finished
time used: 380.552307844162
Extracting results for TXT. It might take a while...
crawled 428 videos
building channel table for TXT
453 finished
time used: 205.49333715438843
Extracting results for TYL. It might take a while...
crawled 411 videos
building channel table for TYL
454 finished
time used: 212.46117877960205
Extracting results for UA. It might take a while...
crawled 357 videos
building channel table for UA
455 finished
time used: 240.8221321105957
Extracting results for UAA. It might take a while...
crawled 300 videos
building channel table for UAA
456 finished
time used: 

KeyboardInterrupt: 

In [72]:
#disconnected, start from 463 URI
for index, row in df.iterrows():
    if index >= 463:
        a = time.time()
        ticker, noise, name = row[0], row[1], row[2]
        if noise == 0:
            search = ticker
        else:
            search = name

        videos = get_videos(search)
        video_lst[row[0]] = videos #store raw datat in dict

        name_list = []
        #build a video table
        print("building channel table for {}".format(ticker))
        for video in videos:
            channel = video['channel']['channel_name']
            link = video['channel']['channel_link']
            if channel not in name_list:
                name_list.append(channel)
                channel_list.append([ticker, name, noise, channel, link])    
            if len(name_list) >= 20:
                break

        b = time.time()
        print(index+1, 'finished')
        print('time used:', b-a)

Extracting results for URI. It might take a while...
crawled 327 videos
building channel table for URI
464 finished
time used: 203.1513171195984
Extracting results for USB. It might take a while...
crawled 480 videos
building channel table for USB
465 finished
time used: 261.57877492904663
Extracting results for Visa. It might take a while...
crawled 534 videos
building channel table for V
466 finished
time used: 266.414911031723


KeyboardInterrupt: 

In [73]:
#disconnected, start from 466 VFC
for index, row in df.iterrows():
    if index >= 466:
        a = time.time()
        ticker, noise, name = row[0], row[1], row[2]
        if noise == 0:
            search = ticker
        else:
            search = name

        videos = get_videos(search)
        video_lst[row[0]] = videos #store raw datat in dict

        name_list = []
        #build a video table
        print("building channel table for {}".format(ticker))
        for video in videos:
            channel = video['channel']['channel_name']
            link = video['channel']['channel_link']
            if channel not in name_list:
                name_list.append(channel)
                channel_list.append([ticker, name, noise, channel, link])    
            if len(name_list) >= 20:
                break

        b = time.time()
        print(index+1, 'finished')
        print('time used:', b-a)

Extracting results for VFC. It might take a while...
crawled 517 videos
building channel table for VFC
467 finished
time used: 257.9469270706177
Extracting results for VIAC. It might take a while...
crawled 407 videos
building channel table for VIAC
468 finished
time used: 245.2539370059967
Extracting results for VLO. It might take a while...
crawled 433 videos
building channel table for VLO
469 finished
time used: 220.97231721878052
Extracting results for VMC. It might take a while...
crawled 383 videos
building channel table for VMC
470 finished
time used: 242.20457482337952
Extracting results for VNO. It might take a while...
crawled 227 videos
building channel table for VNO
471 finished
time used: 216.35070323944092
Extracting results for VRSK. It might take a while...
crawled 182 videos
building channel table for VRSK
472 finished
time used: 204.9187319278717
Extracting results for VRSN. It might take a while...
crawled 161 videos
building channel table for VRSN
473 finished
time 

KeyboardInterrupt: 

In [74]:
#disconnected, start from 487 WMB till the end
for index, row in df.iterrows():
    if index >= 487:
        a = time.time()
        ticker, noise, name = row[0], row[1], row[2]
        if noise == 0:
            search = ticker
        else:
            search = name

        videos = get_videos(search)
        video_lst[row[0]] = videos #store raw data in dict

        name_list = []
        #build a video table
        print("building channel table for {}".format(ticker))
        for video in videos:
            channel = video['channel']['channel_name']
            link = video['channel']['channel_link']
            if channel not in name_list:
                name_list.append(channel)
                channel_list.append([ticker, name, noise, channel, link])    
            if len(name_list) >= 20:
                break

        b = time.time()
        print(index+1, 'finished')
        print('time used:', b-a)

Extracting results for WMB. It might take a while...
crawled 278 videos
building channel table for WMB
488 finished
time used: 218.84243297576904
Extracting results for WMT. It might take a while...
crawled 264 videos
building channel table for WMT
489 finished
time used: 226.206120967865
Extracting results for WRB. It might take a while...
crawled 408 videos
building channel table for WRB
490 finished
time used: 223.61219310760498
Extracting results for WRK. It might take a while...
crawled 383 videos
building channel table for WRK
491 finished
time used: 178.8677179813385
Extracting results for WST. It might take a while...
crawled 400 videos
building channel table for WST
492 finished
time used: 274.14357709884644
Extracting results for WU. It might take a while...
crawled 319 videos
building channel table for WU
493 finished
time used: 206.70943021774292
Extracting results for WY. It might take a while...
crawled 350 videos
building channel table for WY
494 finished
time used: 195.